In [1]:
import sys
sys.path.append('..')

#from kitchen import app, db
#from kitchen.stove.models import Comic, Volume, Story_Arc

from utils.dbobjects import *
from utils.utils import *
from utils.config import read_api_key

from urllib.request import urlretrieve

from comicvine import ComicVineClient
from scanners import Vanilla

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

import requests
import pprint
import functools
import json
import shutil
import pprint as pp
import re
import os

import filecmp
from pathlib import Path
import datetime
import numpy as np
from tqdm.auto import tqdm
from IPython.display import clear_output


ImportError: cannot import name 'read_api_key'

In [9]:
api_key = read_api_key()
cv = ComicVineClient(api_key)

class LibraryParser:
    def __init__(self, library_path):
        self.library_path = Path(library_path).resolve()
        self.n_comics = None
        self.n_volumes = None
        self.folder_paths = []
        self.library_dict = {}
        
    def library_watchdog(self):
        pass
    
    def scan_root_folder(self):
        #folders = list({each.parent for each_ext in ['*.cbr', '*.cbz'] for each in self.library_path.rglob(each_ext)})
        scanner = Vanilla(library_path=self.library_path)
        self.library_dict = scanner.scan()
        
    def volumify(self):
        for each in self.library_dict:            
             for each_type in self.library_dict[each]:
                self.library_dict[each][each_type]['parser'] = VolumeParser(name=each, path=self.library_dict[each][each_type]['path'], paths_in_folder=self.library_dict[each][each_type]['paths_in_folder'])
                
    def parse_library(self):
        [self.library_dict[each_volume][each_type]['parser'].parseVolume() for each_volume in self.library_dict.keys() for each_type in self.library_dict[each_volume].keys()]
        
    def search_volumes(self):
        [self.library_dict[each_volume][each_type]['parser'].searchQuery(api_key=api_key) for each_volume in self.library_dict.keys() for each_type in self.library_dict[each_volume].keys()]
    
    def show_best(self):
        [self.library_dict[each_volume][each_type]['parser'].findBestSearch() for each_volume in self.library_dict.keys() for each_type in self.library_dict[each_volume].keys()]
    
    def confirm_best(self):
        [self.library_dict[each_volume][each_type]['parser'].confirmResult() for each_volume in self.library_dict.keys() for each_type in self.library_dict[each_volume].keys()]
   
    

In [10]:
x = LibraryParser('../data/')
x.scan_root_folder()
x.volumify()

In [ ]:
pp.pprint(x.library_dict)

In [11]:
x.parse_library()

Marvel Masterworks - The Inhumans Vol  01
Infinity Crusade
Infinity Crusade
Vol. 1 Tales to Astonish
Incredible Hulk 600-611
Inhumans v4
Infinity Gauntlet
Infinity Gauntlet Aftermath
Age of Ultron
Thunderbolts Vol. 2
Thunderbolts Vol. 1
X-Men - Legacy
X-Men - Legacy


In [12]:
x.search_volumes()

Fetching from Comic Vine servers
Querying for Marvel Masterworks - The Inhumans Vol  01 2009
Fetching from Comic Vine servers
Querying for Infinity Crusade 2009
Fetching from Comic Vine servers
Querying for Infinity Crusade
Fetching from Comic Vine servers
Querying for Vol. 1 Tales to Astonish 1959
Fetching from Comic Vine servers
Querying for Incredible Hulk 600-611 2009
Fetching from Comic Vine servers
Querying for Inhumans v4
Fetching from Comic Vine servers
Querying for Infinity Gauntlet
Fetching from Comic Vine servers
Querying for Infinity Gauntlet Aftermath 2013
Fetching from Comic Vine servers
Querying for Age of Ultron 2013
Fetching from Comic Vine servers
Querying for Thunderbolts Vol. 2 2006
Fetching from Comic Vine servers
Querying for Thunderbolts Vol. 1 1997
Fetching from Comic Vine servers
Querying for X-Men - Legacy 2009
Fetching from Comic Vine servers
Querying for X-Men - Legacy 2008


In [13]:
x.show_best()

Finding best match
Match Score : 70.0
Finding best match
Best score is only 10.0. Performing deep search on the results.
Match Score : 62.0
Finding best match
Best score is only 0.0. Performing deep search on the results.
Match Score : 72.0
Finding best match
Match Score : 80.0
Finding best match
Match Score : 50.0
Finding best match
Best score is only 0.0. Performing deep search on the results.
Match Score : 100.0
Finding best match
Best score is only 10.0. Performing deep search on the results.
Match Score : 110.0
Finding best match
Match Score : 80.0
Finding best match
Match Score : 70.0
Finding best match
Best score is only 0.0. Performing deep search on the results.
Match Score : 66.0
Finding best match
Best score is only 0.0. Performing deep search on the results.
Match Score : 60.0
Finding best match
Match Score : 50.0
Finding best match
Match Score : 80.0


In [14]:
x.confirm_best()


No. of issues within folder : 69
Range of years within folder : {2008, 2009, 2010, 2011, 2012}


In [2]:
class ComicParser(): #Parser object to extract name, first year and issue number.
    def __init__(self, path):
        self.comic_name = None
        self.comic_year = None
        self.issue_number = None
        self.comic_path = path
        self.comic_metadata = {}
        self.comic_initiated = False

    def parseComic(self):
        self.comic_name = self.comic_path.name #os.path.splitext(os.path.basename(self.comic_path))[0]
        years = re.findall(r'\(([12]\d{3})', self.comic_name)
        if years:
            year_within_brackets = years[0]
            if (len(year_within_brackets) == 4): #Stupid way of doing but good for now.
                self.comic_year = int(year_within_brackets)
            else:
                self.comic_year = 0
        self.issue_number = re.findall(r"(\d+\.?\d?[a-zA-Z]{0,3}?)(?:\s*\(of|\s*\([12]\d{3}\))", self.comic_name) #Get the digits or digits with point number (19.1) or digits followed by letters (19.INH) either before "(of" or before a year. 

In [8]:
class VolumeParser:
    def __init__(self, name=None, path=None, paths_in_folder=None):
        self.volume_name = name
        self.volume_path = path
        self.volumeDict = {'paths_in_folder' : paths_in_folder,
                          'assets_in_folder': {}}
        self.asset_count = 0
        self.comic_years = None
        
        self.volume_initiated = False
        self.update_index = 0
        self.volumeTable = None
        
        
        self.searchReturned = None
        self.HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:7.0) '
                     'Gecko/20130825 Firefox/36.0'}
        self.params = {'api_key':api_key, 'format': 'json'} 
    
    
    def scan_volume(self): #Adds path and volume name to the parser object.
        """
        Need to make sure, only if the parser is not initiated by a library, we need to add pathlib actions here. We can use None passes.
        """
        
        pass
    
    def parseVolume(self): # Also parses comic. 
        if '(' in self.volume_name:
            self.volume_name = self.volume_name.split(' (')[0].strip()
        else:    
            self.volume_name = self.volume_name
        
        print(self.volume_name)
        self.volumeDict['from_file_volume_name'] = self.volume_name
        
        for each_asset in self.volumeDict['paths_in_folder']:
            comic = ComicParser(each_asset) #convert to path join volume path and entry
            comic.parseComic()            
            self.volumeDict['assets_in_folder'].update({comic.comic_name : comic})
                                
#         print(str(len(self.volumeDict['comics_in_folder']))+' comics added in volume ' + self.volume_name +'.')    
        self.asset_count = len(self.volumeDict['assets_in_folder']) 
        self.comic_years = {self.volumeDict['assets_in_folder'][key].comic_year for key in self.volumeDict['assets_in_folder'].keys() if self.volumeDict['assets_in_folder'][key].comic_year}
        del(self.volumeDict['paths_in_folder'])
    
    def searchQuery(self,api_key): #Search ComicVine Server for metadata
        print("Fetching from Comic Vine servers")
        years = [self.volumeDict['assets_in_folder'][key].comic_year for key in self.volumeDict['assets_in_folder'].keys() if self.volumeDict['assets_in_folder'][key].comic_year]        
        searchtext = self.volumeDict['from_file_volume_name']
        if self.comic_years:
            searchtext += (' '+str(min(self.comic_years)))
        print('Querying for '+searchtext)
        firstresponse = cv.search(searchtext, resources=['volume']) #Search for volume name with year
        self.searchReturned = firstresponse.results
        
    def findBestSearch(self, deep_check=True):
        from fuzzywuzzy import fuzz
        """
        Finding best search among Volumes. 2 main information considered 
        here are number of assets in folder folder (which is assumed to be 
        total number of issues in that volume) and year range obtained 
        from comic file names.        
        
        First Result | No. of Issues | Year | Score | Result
              o               o          o      80      o
              o               X          o      70      o
              X               o          o      50      o
              o               o          X      40      X -
              X               X          o      40      X |
              o               X          X      30      X |-- Fuzzy search for these guys
              X               o          X      10      X -
        
        """
        print("Finding best match")
        
        existing_comics = list(self.volumeDict['assets_in_folder'].keys())
        
        best_index = 0
        for indx, each_result in enumerate(self.searchReturned):    
            if indx is 0:
                each_result['match_score'] = 30.0 #First result returned is the best result.
            elif indx is not 0:
                each_result['match_score'] = 0.0
            if self.asset_count == each_result['count_of_issues']:
                each_result['match_score'] += 10.0 
            if self.comic_years:    
                if (min(self.comic_years)<=int(each_result['start_year'])<=max(self.comic_years)):
                    each_result['match_score'] +=40.0                                            
            if each_result['match_score']>=np.array(self.searchReturned)[best_index]['match_score']: #Also assumed that search results are coming back sorted in relavance.
                best_index = indx
        self.volumeDict['best_search']=np.array(self.searchReturned)[best_index]
        if deep_check and self.volumeDict['best_search']['match_score']<=40: #Deep fuzzy search over titles in volume and existing comics.
            print('Best score is only '+ str(each_result['match_score'])+'. Performing deep search on the results.')
            for indx, each_result in enumerate(self.searchReturned):    
                detailresponse = requests.get(each_result['api_detail_url'], headers=self.HEADERS, params=self.params)
                titles_in_vol = [each['name'] for each in detailresponse.json()['results']['issues']]
                each_result['match_score'] +=fuzz.token_set_ratio(titles_in_vol, existing_comics) #Levenshtein Distance
                if each_result['match_score']>=np.array(self.searchReturned)[best_index]['match_score']: #Also assumed that search results are coming back sorted in relavance.
                    best_index = indx

        self.volumeDict['best_search']=np.array(self.searchReturned)[best_index]
        print('Match Score : '+ str(self.volumeDict['best_search']['match_score']))        
    
    def confirmResult(self):
        clear_output()
        print_com_meta(self.volumeDict['best_search'], 'Best Result:')
        confirmation = query_yes_no("Is the information correct?")
        choice = None
        while True:
            clear_output()
            print('\nNo. of issues within folder : '+str(self.asset_count))
            print('Range of years within folder : '+str(self.comic_years))
            if not confirmation: 
                for idx, each in enumerate(self.searchReturned):
                    print_com_meta(each, idx+1)
                choice = sanitised_input("Please make a manual choice between 1-10 : ", int, 1,10)
                print_com_meta(self.searchReturned[choice-1], 'Your Choice : ')
                confirmation = query_yes_no("Is the information correct?")
            else:
                if choice:
                    self.volumeDict['best_search'] = self.searchReturned[choice-1]
                break
    
    def fetchVolumeMetadata(self):
        detailresponse = requests.get(self.volumeDict['best_search']['api_detail_url'], headers=self.HEADERS, params=self.params)
        fetched_volume = detailresponse.json()
        self.volumeDict['detailed_meta'] = fetched_volume['results']        
        #Image Directory
        orig_directory = './resources/Images/VolumeArt/'
        img_name = os.path.basename(self.volumeDict['detailed_meta']['image']['original_url'])
        img_path = os.path.join(orig_directory,img_name)
        #Fetch Image
        urlretrieve(self.volumeDict['detailed_meta']['image']['original_url'], img_path)        
        self.volume_initiated = True
        volume = Volume(
                        id = self.volumeDict['detailed_meta']['id'],
                        name = self.volumeDict['detailed_meta']['name'],
                        aliases = self.volumeDict['detailed_meta']['aliases'],
                        count_of_issues = self.volumeDict['detailed_meta']['count_of_issues'],
                        date_added = self.volumeDict['detailed_meta']['date_added'],
                        date_last_updated = self.volumeDict['detailed_meta']['date_last_updated'],
                        deck = self.volumeDict['detailed_meta']['deck'],
                        description = re.sub(r'<[^>]*>', '', self.volumeDict['detailed_meta']['description']), #remove all within angle brackets
                        publisher = self.volumeDict['detailed_meta']['publisher']['name'], #Later to be changed to publisher object
                        start_year = self.volumeDict['detailed_meta']['start_year'],
                        comicvine_api_detail_url = self.volumeDict['detailed_meta']['api_detail_url'],
                        comicvine_image = self.volumeDict['detailed_meta']['image']['original_url'], #image
                        comicvine_site_detail_url = self.volumeDict['detailed_meta']['site_detail_url'],
                        local_path = self.volume_path,
                        local_image_path = img_path,
                        #character_credits = Column(String)
                        #concept_credits = Column(String)
                        #team_credits = Column(String)
                        #location_credits = Column(String)
                        #object_credits = Column(String)
                        #person_credits = Column(String)
                        )
        self.volumeTable = volume        
        
    def fetchComicsMetadata(self, api_key):
        comic_fetch_loop = tqdm(self.volumeDict['detailed_meta']['issues'])
        comicObjects = []
        issue_directory = './resources/Images/IssueArt/'                        
        for each_issue in comic_fetch_loop:
            per_issue_response = requests.get(each_issue['api_detail_url'], headers=self.HEADERS, params=self.params).json()['results']
            img_name = os.path.basename(per_issue_response['image']['original_url'])
            img_path = os.path.join(issue_directory,img_name)
            #Fetch Image if doesnt exist
            if not os.path.isfile(img_path):
                urlretrieve(per_issue_response['image']['original_url'], img_path) 
            
            comic = Comic(id = per_issue_response['id'],
                          name = per_issue_response['name'],
                          aliases = per_issue_response['aliases'],
                          deck = per_issue_response['deck'],
                          description = re.sub(r'<[^>]*>', '', per_issue_response['description']),
                          cover_date = per_issue_response['cover_date'],
                          date_added = per_issue_response['date_added'],
                          date_last_updated = per_issue_response['date_last_updated'],
                          issue_number = per_issue_response['issue_number'],
                          comicvine_api_detail_url = per_issue_response['api_detail_url'],
                          comicvine_image = per_issue_response['image']['original_url'],
                          local_image_path = img_path
            )
            comicObjects.append(comic)
            comic_fetch_loop.set_postfix(Status='Added '+str(comic.name))            
        self.volumeTable.comics=set(comicObjects)
    
    def fetch_arc_data(self):
        pass
        arc_objects = []
        for each_comic in self.volumeTable.comics:
            for each_arc in each_comic['story_arc_credits']:
                arc = Story_Arc()
    
    def commit_to_db(self, dbpath='sqlite:///comicdb.db'):
        
        app.config['SQLALCHEMY_DATABASE_URI'] = dbpath
        #db = SQLAlchemy(app)
        
        
        #engine = create_engine(dbpath, echo=False)
        #Base.metadata.create_all(engine)
        #Session = sessionmaker(bind=engine)
        #session = Session()
        db.session.add(self.volumeTable)
        
        try:
            db.session.commit()
        except IntegrityError:
            db.session.rollback()
        
        db.session.close()
        engine.dispose()

            
#             for key in self.volumeDict['comics_in_folder'].keys():
#                 if self.volumeDict['comics_in_folder'][key].issue_number == per_issue_response['issue_number'] and self.volumeDict['comics_in_folder'][key].comic_initiated == False:
#                     self.volumeDict['comics_in_folder'][key].comic_metadata = per_issue_response
#                     self.volumeDict['comics_in_folder'][key].comic_initiated = True

In [90]:
arc = requests.get('https://comicvine.gamespot.com/api/issue/4000-504345', headers=VolumeParser().HEADERS, params=VolumeParser().params)

In [91]:
list(arc.json()['results'])

['aliases',
 'api_detail_url',
 'character_credits',
 'character_died_in',
 'concept_credits',
 'cover_date',
 'date_added',
 'date_last_updated',
 'deck',
 'description',
 'first_appearance_characters',
 'first_appearance_concepts',
 'first_appearance_locations',
 'first_appearance_objects',
 'first_appearance_storyarcs',
 'first_appearance_teams',
 'has_staff_review',
 'id',
 'image',
 'issue_number',
 'location_credits',
 'name',
 'object_credits',
 'person_credits',
 'site_detail_url',
 'store_date',
 'story_arc_credits',
 'team_credits',
 'team_disbanded_in',
 'volume']

In [74]:
char = requests.get(arc.json()['results']['character_credits'][0]['api_detail_url'], headers=VolumeParser().HEADERS, params=VolumeParser().params)
char.json()['results']

{'aliases': 'Him\r\nMagus\r\nWarlock\r\nGod Slayer\r\nGolden Surfer\r\nGoldie Locks\r\nAdam\r\nThe Perfect Human\r\nThe Avatar of Life\r\nAdam Magus',
 'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-6805/',
 'birth': None,
 'character_enemies': [{'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-44289/',
   'id': 44289,
   'name': 'Akhenaten',
   'site_detail_url': 'https://comicvine.gamespot.com/akhenaten/4005-44289/'},
  {'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-10964/',
   'id': 10964,
   'name': 'Annihilus',
   'site_detail_url': 'https://comicvine.gamespot.com/annihilus/4005-10964/'},
  {'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-118570/',
   'id': 118570,
   'name': 'Anwen Bakian',
   'site_detail_url': 'https://comicvine.gamespot.com/anwen-bakian/4005-118570/'},
  {'api_detail_url': 'https://comicvine.gamespot.com/api/character/4005-148102/',
   'id': 148102,
   'name': 'Arokine',
   '